In [1]:
import os
import sys
import pickle
import csv
import copy

import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [3]:
sys.path.append("../../../tfobjdetect/lib")
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

../../../tfobjdetect/lib/object_detection/utils/visualization_utils.py:25: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/alireza/home_at_filer2/tools/anaconda3/envs/py3tf/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/alireza/home_at_filer2/tools/anaconda3/envs/py3tf/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/alireza/home_at_filer2/tools/anaconda3/envs/py3tf/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/alireza/home_at_filer2/tools/anaconda3/envs/py3tf/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch

## Loading and preparing label maps

In [4]:
with open('../../temp/label_matching_coco_oi_ws_300.pkl', 'rb') as fin:
    labelmap = pickle.load(fin)

In [5]:
idx2mid_coco = labelmap['idx2mid_coco']
idx2mid_oi = labelmap['idx2mid_oi']
idx2mid_ws = labelmap['idx2mid_ws']
mid2idx_coco = labelmap['mid2idx_coco']
mid2idx_oi = labelmap['mid2idx_oi']
mid2idx_ws = labelmap['mid2idx_ws']
mid2name_coco = labelmap['mid2name_coco']
mid2name_oi = labelmap['mid2name_oi']
mid2name_ws = labelmap['mid2name_ws']
name2mid_coco = labelmap['name2mid_coco']
name2mid_oi = labelmap['name2mid_oi']
name2mid_ws = labelmap['name2mid_ws']
common_mids_coco = labelmap['common_mids_coco']
common_mids_ws = labelmap['common_mids_ws']


In [6]:
coco_mids = set(mid2idx_coco.keys())
oi_mids = set(mid2idx_oi.keys())
ws_mids = set(mid2idx_ws.keys())


In [7]:
mid2name_all = {}
for key in coco_mids:
    mid2name_all[key] = mid2name_coco[key]
for key in oi_mids:
    mid2name_all[key] = mid2name_oi[key]
for key in ws_mids:
    mid2name_all[key] = mid2name_ws[key]

In [8]:
with open('../../temp/all_labels_coco_oi_ws_300.pkl', 'rb') as fin:
    all_labels = pickle.load(fin)

In [9]:
with open('../../results/det_results_postproc_10.pkl', 'rb') as fin:
    det_results_sorted = pickle.load(fin)

## Choosing examples for each class to visualize

In [10]:
label_to_img = {key: [] for key in all_labels}
for key, val in det_results_sorted.items():
    for ii in range(val['num_detections']):
        if val['detection_scores'][ii] < 0.35:
            continue
        label_to_img[val['detection_classes'][ii]].append(key)

In [19]:
idx = np.argsort([-len(label_to_img[label]) for label in all_labels])

In [21]:
for i in idx:
    print(all_labels[i], mid2name_all[all_labels[i]], len(label_to_img[all_labels[i]]))

/m/01g317 Person 3800
/m/04yx4 Man 1834
/m/0dzct Human face 1634
/m/03bt1vf Woman 794
/m/05s2s Plant 407
/m/0k4j Car 247
/m/0jbk Animal 244
/m/0c_jw Furniture 242
/m/0hf58v5 Luggage and bags 231
/m/012t_z Businessperson 224
/m/0cgh4 Building 216
/m/01n5jq Poster 193
/m/01xr66 Spokesperson 155
/m/01prls Land vehicle 150
/m/02wbm Food 120
/m/0jyfg Glasses 120
/m/0cmf2 Airplane 112
/m/07r04 Truck 111
/m/07yv9 Vehicle 105
/m/0dwx7 Logo 101
/m/07s6nbt Text 98
/m/0zvk5 Helmet 95
/m/04_tb Map 95
/m/01fnck Waste 93
/m/07k1x Tool 93
/m/050k8 Mobile phone 93
/m/019jd Boat 81
/m/03120 Flag 80
/m/07c52 Television 72
/m/07jdr Train 62
/m/035y33 Official 61
/m/02_41 Fire 60
/m/07cmd Tank 59
/m/06q40 Smoke 59
/m/03jm5 House 54
/m/05r655 Girl 54
/m/0bt9lr Dog 53
/m/015h_t Human beard 51
/m/01bl7v Boy 41
/m/0bs7_0t Electronic device 41
/m/03qtwd Crowd 40
/m/09x0r Speech 40
/m/09nm_ World 36
/m/01xgg_ Medical equipment 33
/m/0hg7b Microphone 33
/m/02_n6y Goggles 32
/m/01fdzj Tower 30
/m/08qrwn Rubble 29

In [28]:
select_labels = ['/m/01fnck', '/m/019jd', '/m/03120', '/m/02_41', '/m/07cmd', '/m/06q40', '/m/03qtwd', '/m/09x0r', '/m/01xgg_', '/m/08qrwn', '/m/012n4x', '/m/02p16m6', '/m/02lbcq', '/m/09ct_', '/m/0g54v5d', '/m/0bg2p', '/m/0ct4f', '/m/01nd_n', '/m/09rvcxw', '/m/01nl4x', '/m/04ctx', '/m/04ylt', '/m/0gvss07', '/m/01rzcn', '/m/06nrc', '/m/0cyfs', '/m/0f5lx', '/m/04zjc', '/m/0dhz0', '/m/01lcw4', '/m/03htg', '/m/0gxl3', '/m/01bq8v', '/m/0lt4_', '/m/02gzp', '/m/02yjc', '/m/012n7d', '/m/0jb3']

## Visualizing Results

In [12]:
mid2name_extended = {}
extended_classes = set()
for key in coco_mids:
    for model, abv in [('coco', 'CO'), ('oi', 'OI'), ('ws', 'WS'), ('coco/J', 'CO/J'), ('oi/J', 'OI/J'), ('ws/J', 'WS/J')]:
        extended_classes.add(key + '/' + model)
        mid2name_extended[key + '/' + model] = mid2name_coco[key] + f' ({abv})'
for key in oi_mids:
    for model, abv in [('coco', 'CO'), ('oi', 'OI'), ('ws', 'WS'), ('coco/J', 'CO/J'), ('oi/J', 'OI/J'), ('ws/J', 'WS/J')]:
        extended_classes.add(key + '/' + model)
        mid2name_extended[key + '/' + model] = mid2name_oi[key] + f' ({abv})'
for key in ws_mids:
    for model, abv in [('coco', 'CO'), ('oi', 'OI'), ('ws', 'WS'), ('coco/J', 'CO/J'), ('oi/J', 'OI/J'), ('ws/J', 'WS/J')]:
        extended_classes.add(key + '/' + model)
        mid2name_extended[key + '/' + model] = mid2name_ws[key] + f' ({abv})'
extended_classes = list(extended_classes)
mid2idx_extended = {}
for i, key in enumerate(extended_classes):
    mid2idx_extended[key] = i
category_index_extended = {mid2idx_extended[key]:{'id': mid2idx_extended[key], 'name': mid2name_extended[key]} for key in mid2name_extended}

In [13]:
det_results_forviz = copy.deepcopy(det_results_sorted)

In [14]:
for key in det_results_forviz:
    for i in range(len(det_results_forviz[key]['detection_classes'])):
        label = det_results_forviz[key]['detection_classes'][i] + '/' + det_results_forviz[key]['detection_models'][i]
        det_results_forviz[key]['detection_classes'][i] = mid2idx_extended[label]


In [15]:
print(det_results_forviz[key]['detection_classes'])

[2554, 3896, 339, 4405, 1676, 3657, 1076, 3878, 1076, 1866, 1676, 3837, 1929, 3776, 2460, 1352, 69, 3550, 3103, 1411, 2578, 636, 252, 505, 40, 2321, 505, 4081, 2070, 2178, 2960, 3823, 3690, 2271, 903, 501, 932]


In [16]:
def preproc(im):
    target_size = 400
    max_size = 1024
    im_size_min = np.min(im.shape[0:2])
    im_size_max = np.max(im.shape[0:2])
    im_scale = float(target_size) / float(im_size_min)
    # Prevent the biggest axis from being more than MAX_SIZE
    if np.round(im_scale * im_size_max) > max_size:
        im_scale = float(max_size) / float(im_size_max)
    im = cv2.resize(im, None, None, fx=im_scale, fy=im_scale,
                    interpolation=cv2.INTER_LINEAR)    
    return im

In [26]:
def show(imgid, thresh):
    filename = '../../../../data/seedling/seedling_part1/jpg/' + imgid
    print(imgid)
    with open(filename, 'rb') as fin:
        _ = fin.read(1024)
        imgbin = fin.read()
    imgbgr = cv2.imdecode(np.fromstring(imgbin, dtype='uint8'), cv2.IMREAD_COLOR)
    image_np = imgbgr[:,:,[2,1,0]]
    image_np = preproc(image_np)
    vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      np.asarray(det_results_forviz[imgid]['detection_boxes_normalized']),
      det_results_forviz[imgid]['detection_classes'],
      det_results_forviz[imgid]['detection_scores'],
      category_index_extended,
      instance_masks=det_results_forviz[imgid].get('detection_masks'),
      use_normalized_coordinates=True,
      min_score_thresh=thresh,
      line_thickness=2)
    plt.figure(figsize=(12, 8))
    plt.imshow(image_np)
    plt.axis('off')
    plt.show()    

In [29]:
for label in select_labels:
    if len(label_to_img[label]) == 0:
        continue
    print(mid2name_all[label])
    imgid = np.random.choice(label_to_img[label])
    w = interactive(show, 
        thresh=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.35, continuous_update=False), 
        imgid=fixed(imgid)
    )
    w.children[-1].layout.height = '600px'
    display(w)

Waste


interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

Boat


interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

Flag


...

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

Ambulance


interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

Assault rifle


interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

In [30]:
for label in select_labels:
    if len(label_to_img[label]) == 0:
        continue
    print(mid2name_all[label])
    imgid = np.random.choice(label_to_img[label])
    w = interactive(show, 
        thresh=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.35, continuous_update=False), 
        imgid=fixed(imgid)
    )
    w.children[-1].layout.height = '600px'
    display(w)

Waste


interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

Boat


interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

Flag


...

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

Ambulance


interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

Assault rifle


interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

In [31]:
for label in select_labels:
    if len(label_to_img[label]) == 0:
        continue
    print(mid2name_all[label])
    imgid = np.random.choice(label_to_img[label])
    w = interactive(show, 
        thresh=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.35, continuous_update=False), 
        imgid=fixed(imgid)
    )
    w.children[-1].layout.height = '600px'
    display(w)

Waste


interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

Boat


interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

Flag


...

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

Ambulance


interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

Assault rifle


interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

In [32]:
for label in select_labels:
    if len(label_to_img[label]) == 0:
        continue
    print(mid2name_all[label])
    imgid = np.random.choice(label_to_img[label])
    w = interactive(show, 
        thresh=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.35, continuous_update=False), 
        imgid=fixed(imgid)
    )
    w.children[-1].layout.height = '600px'
    display(w)

Waste


interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

Boat


interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

Flag


...

interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

Ambulance


interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…

Assault rifle


interactive(children=(FloatSlider(value=0.35, continuous_update=False, description='thresh', max=1.0, step=0.0…